In [1]:
"""
LASSO Regression: y = w0*x0 + w1*x1 + w2*x2 + ... + wD*xD
Objective: Estimate w0, w1, w2 ... wD given x1, x2 ... xD and y
where xN = input feature, x0 = 1.0
      y  = output
subject to L1 penalty
"""

# Imports
import numpy as np
import copy

# Functions
def get_data(data_frame, features, output):
    """
    Purpose: Extract features and prepare a feature matrix
             Set the first feature x0 = 1
    Input  : Original Dataframe, list of feature variables, output variable
    Output : Feature matrix array, output array
    """
    data_frame['constant'] = 1.0
    features = ['constant'] + features
    features_matrix = np.array(data_frame[features])
    if output != None:    
        output_array = np.array(data_frame[output])
    else:
        output_array = []   
    return(features_matrix, output_array.reshape((len(output_array))))

def normalize(features):
    """
    Purpose: Normalize feature matrix, each column of the matrix is a feature
    Input  : Unnormalized feature matrix
    Output : Normalized feature matrix, feature norms
    """
    norms = np.linalg.norm(features, axis=0)
    normalized_features = features/norms
    return (normalized_features, norms)

def predict(feature_matrix, weights):
    """
    Purpose: Compute predictions by multiplying the feature matrix with
             the estimated weights
    Input  : Feature matrix array, weight vector
    Output : Product of feature matrix array and weight vector
    """
    predictions = feature_matrix.dot(weights)
    return(predictions)

def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    """
    Purpose: Compute the descent step for one feature
    Input  : Feature index, normalized feature matrix, output,
             feature weights and L1_penalty
    Output : Descent step for feature
    """
    predictions = feature_matrix.dot(weights)
    rho = (feature_matrix[:, i].T).dot(output - predictions + (weights[i] * feature_matrix[:, i]))
    if i==0:
        new_weight = rho
    elif rho < (-l1_penalty/2.0):
        new_weight = rho + (l1_penalty/2.0)
    elif rho > (l1_penalty/2.0):
        new_weight = rho - (l1_penalty/2.0)
    else:
        new_weight = 0.0
    return new_weight

def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    """
    Purpose: Perform cyclical coordinate descent
    Input  : Normalized feature matrix, output, initial weights,
             L1_penalty and tolerance for stopping the process
    Output : Final weights after the convergence of the coordinate
             descent procedure
    """
    D = feature_matrix.shape[1]
    weights = copy.copy(initial_weights)
    change = np.zeros(initial_weights.shape)
    converged = False
    
    while not converged:
        # Evaluate over all features
        for idx in range(D):
            # New weight for feature
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            # Compute change in weight for feature
            change[idx] = np.abs(new_weight - weights[idx])
            # assign new weight
            weights[idx] = new_weight
        # Maximum change in weight, after all changes have been computed
        max_change = max(change)
        if max_change < tolerance:
            converged = True
    return weights

def fit(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    """
    Purpose: Wrapper for cyclical coordinate descent function
    Input  : Feature matrix array, initial weight vector, output vector,
             tolerance value, l1_penalty
    Output : Estimated weight vector
    """
    weights = lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance)
    return(weights)

def get_residual_sum_of_squares(feature_matrix, weights, output):
    """
    Purpose: Compute Residual Sum of Squares (RSS)
    Input  : Feature matrix, weight vector, output vector
    Output : Residual sum of squares = sum((actual output (y) - predicted output)^2)
    """
    predictions = predict(feature_matrix, weights)
    residual = np.sum((predictions - output) ** 2)
    return(residual)

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int,
              'sqft_living15':float, 'grade':int, 'yr_renovated':int,
              'price':float, 'bedrooms':float, 'zipcode':str,
              'long':float, 'sqft_lot15':float, 'sqft_living':float,
              'floors':float, 'condition':int, 'lat':float, 'date':str,
              'sqft_basement':int, 'yr_built':int, 'id':str,
              'sqft_lot':int, 'view':int}

In [5]:
import pandas as pd
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

In [37]:
features = ['sqft_living', 'bedrooms']
target = ['price']
# Extract and normalize feature matrix
feature_matrix, output = get_data(sales, features, target)
norm_feature_matrix, norms =normalize(feature_matrix)
print norm_feature_matrix

[[ 0.00680209  0.00353021  0.00583571]
 [ 0.00680209  0.00768869  0.00583571]
 [ 0.00680209  0.00230361  0.00389048]
 ..., 
 [ 0.00680209  0.00305154  0.00389048]
 [ 0.00680209  0.00478673  0.00583571]
 [ 0.00680209  0.00305154  0.00389048]]


In [38]:
0.00680209 + 4*0.00353021 + 0.00583571

0.02675864

In [36]:
weights=np.array([1.,4.,1.])
predictions = feature_matrix.dot(weights)
print predictions
rho = (norm_feature_matrix[:, 1].T).dot(output - predictions + (weights[1] * norm_feature_matrix[:, 1]))

[  4724.  10284.   3083. ...,   4083.   6404.   4083.]


In [35]:
rho/2

43300913.979649693

In [8]:
init_weights = np.zeros((len(features)+1))
l1_penalty = 1.0e7
tolerance = 1.0

In [9]:
model =fit(norm_feature_matrix, output, init_weights, l1_penalty, tolerance)
RSS =get_residual_sum_of_squares(norm_feature_matrix, model, output)

In [10]:
RSS

1630492476715386.5

In [11]:
model

array([ 21624997.95951909,  63157247.20788956,         0.        ])

In [13]:
# Read house sales train and test data
train_data = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
# Select features and output
features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
            'floors', 'waterfront', 'view', 'condition', 'grade',
            'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
target = ['price']

In [14]:
train_feature_matrix, train_output = get_data(train_data, features, target)
norm_train_feature_matrix, norm_train = normalize(train_feature_matrix)
test_feature_matrix, test_output = get_data(train_data, features, target)

In [16]:
init_weights = np.zeros((len(features)+1))
l1_penalty = 1.0e7
tolerance = 1.0
weights1e7 = fit(norm_train_feature_matrix, train_output, init_weights, l1_penalty, tolerance)

In [18]:
init_weights = np.zeros((len(features)+1))
l1_penalty = 1.0e8
tolerance = 1.0
weights1e8 = fit(norm_train_feature_matrix, train_output, init_weights, l1_penalty, tolerance)


In [20]:
init_weights = np.zeros((len(features)+1))
l1_penalty = 1.0e4
tolerance = 5.0e5
weights1e4 = fit(norm_train_feature_matrix, train_output, init_weights, l1_penalty, tolerance)

In [21]:
norm_weights1e7 = weights1e7/norm_train
norm_weights1e8 = weights1e8/norm_train
norm_weights1e4 = weights1e4/norm_train

In [23]:
RSS_1e7 = get_residual_sum_of_squares(test_feature_matrix, norm_weights1e7, test_output)
RSS_1e8 = get_residual_sum_of_squares(test_feature_matrix, norm_weights1e8, test_output)
RSS_1e4 = get_residual_sum_of_squares(test_feature_matrix, norm_weights1e4, test_output)

In [24]:
RSS_1e7

1231595575158379.5

In [25]:
RSS_1e8

2375761861767301.5

In [26]:
RSS_1e4

990207356732102.62

In [27]:
norm_weights1e7

array([  1.85285530e+05,   0.00000000e+00,   0.00000000e+00,
         1.61317458e+02,   0.00000000e+00,   0.00000000e+00,
         2.87664705e+05,   6.91937041e+04,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00])

In [28]:
norm_weights1e8

array([ 539366.62793373,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ,       0.        ,
             0.        ,       0.        ])

In [29]:
norm_weights1e4

array([  5.95871771e+05,  -4.80336244e+04,   4.30892643e+04,
         3.12732803e+02,  -3.46078585e-01,  -2.01432664e+04,
         5.62133764e+05,   6.72816325e+04,   1.09255888e+04,
         1.40325598e+04,  -6.07214159e+01,  -7.35796867e+01,
        -3.25079490e+02,   5.26011603e+01])

In [31]:
test_feature_matrix.shape

(17384L, 14L)